# 集成方法

可以将不同的分类器组合起来，而这种组合结果就被称为<span style="color: red">集成方法（ensemble method）或元算法（meta-algorithm）</span>。使用集成方法时会有多种形式：
+ 可以是不同算法的集成
+ 同一算法在不同设置的集成
+ 数据集不同部分分配给不同分类器之后的集成

> 元算法：是对其他算法进行组合的一种方式。

> 思想：三个臭皮匠顶个诸葛亮

### 1 基于数据集多重抽样的分类器

#### bagging: 基于数据随机抽样的分类器构建方法

<span style='color: red'>自举汇聚法（bootstrap aggregating）</span>，也称为<span style='color: red'>bagging方法</span>，是在原始数据集选择S次后得到S个新数据集的一种技术。新数据集和原始数据集的大小相等。每个数据集都是通过在原始数据集中随机选择一个样本来进行替换而得到的。这里的替换意味着可以多次选择同一个样本。这就允许新数据集可以有重复的值，而原始数据集的某些值在新集合中则不会出现。

S个数据集就建立好之后，将某个学习算法分别作用于每个数据集就得到了S个分类器。当对新数据进行分类时，就可以应用这S个分类器进行分类。选择分类器投票结果中最多的类别作为最后的分类结果。

还有一些更先进的方法，如随机森林（random forest）。

#### 提升方法（boosting）
<span style='color: red'>boosting</span>是一种与bagging很类似的技术。不论是在boosting还是bagging中，所使用的多个分类器的类型都是一致的。但在boosting中，不同的分类器时通过串行训练得到的，每个新分类器根据已训练出的分类器的性能来进行训练。boosting是通过集中关注被已有分类器错分的那些数据来获得新的分类器。

由于boosting分类的结果是基于所有分类器的加权求和结果的，因此boosting和bagging不太一样。bagging中的分类器权重是相等的，而boosting中的分类器权重并不想等，每个权重代表的是其对应分类器在上一轮迭代中的成功度。

在概率近似正确(probably approximately correct, PCA)的学习框架中，一个概念（一个类），如果存在一个多项式的学习算法能够学习它，并且正确率很高，那么就称这个概念是<span style='color: red'>强可学习（strongly learnable）</span>的；一个概念，如果存在一个多项式的学习算法能够学习它，学习的正确率仅比随机猜测略好，那么就称这个概念是<span style='color: red'>弱可学习（weakly learnable）</span>的。

后来证明**强可学习和弱可学习是等价的，也就是说，在PCA学习的框架下，一个概念是强可学习的充分必要条件是这个概念是弱可学习的。**

在学习中，如果已经发现了弱可学习算法，可以将它提升（boost）为强可学习算法。提升的方法很多，最具代表性的就是<span style='color: red'>**Adaboost算法（AdaBoost algorithm）**</span>。

对分类问题而言，给定一个训练集，求比较粗糙的分类规则（弱分类器）要比求精确的分类规则（强分类器）容易。提升方法就是从弱分类算法出发，反复学习，得到一系列的弱分类器（有称为基本分类器），然后组合这些弱分类器，构成一个强大的分类器。大多数的提升方法都是改变训练数据的概率分布（训练数据的权值分布），针对不同的训练数据分布调用弱学习算法学习一系列分类器。

**对提升方法来说，有两个问题**
1. 在每一轮中R如何改变训练数据的权重或概念分布

    提高那些被前一轮弱分类器错误分类样本的权值，而降低那些被正确分类样本的权值。这样，那些没有得到正确分类的数据，由于其权值的加大而受到后一轮的弱分类器的更大关注。
2. 如何将弱分类器组合成一个强分类器

    AdaBoost采取加权多数表决的方法。具体地，加大分类误差率小的弱分类器的权值，使其在表决中起较大的作用，减小分类误差率大的弱分类器的权值，使其在表决中起较小的作用。
    
AdaBoost的巧妙之处就在于将这些想法自然且有效地实现在同一种算法里。

### 2 AdaBoost算法

假设给定一个二分类的训练数据集：
$$T = \{(x_1,y_1), (x_2,y_2), \cdots, (x_N,y_N)\}$$

其中每个样本点由实例与标记组成。实例$x_i\in \chi \subseteq R^n$，标记$y_i\in \gamma =\{-1,+1\}$，其中$\chi$是实例空间，$\gamma$是标记集合。

<span style="color: red">**AdaBoost算法**</span>

**输入**：训练数据集T；弱学习算法

**输出**：最终分类器$G(x)$

1.初始化训练数据的权值分布：$$D_1 = (w_{11}, \cdots, w_{1i}, \cdots, w_{1N}), w_{1i}=\frac{1}{N}, i=1, 2, \cdots, N$$

2.对$m=1, 2,\cdots, M$

&emsp;&emsp;(a) 使用具有权值分布$D_m$的训练数据集学习，得到基本分类器
$$G_m(x): \chi \rightarrow \{-1,+1\}$$
&emsp;&emsp;(b) 计算$G_m(x)$在数据集上的分类误差率
$$e_m = P(G_m(x_i)\neq y_i) = \sum_{i=1}^N w_{mi}I(G_m(x_i)\neq y_i)$$

&emsp;&emsp;(c) 计算$G_m(x)$的系数 $$ \alpha_m=\frac{1}{2}ln\frac{1-e_m}{e_m}$$

&emsp;&emsp;(d) 更新训练数据集的权值分布
$$D_{m+1}=(w_{m+1,1},\cdots,w_{m+1,i},\cdots,w_{m+1,N})$$

$$w_{m+1,i} = \frac{w_{mi}}{Z_m}exp(-\alpha_my_iG_m(x_i))$$

&emsp;&emsp;&emsp;&emsp;$Z_m$是规范化因子，它使$D_{m+1}$成为一个概念分布$$Z_m=\sum_{i=1}^N{w_{mi}exp(-\alpha_my_iG_m(x_i))}$$
3.构建基本分类器的线性组合
$$f(x)=\sum_{m=1}^M{\alpha_mG_m(x)}$$
得到最终分类器$$G(x) = sign(f(x))=sign\left(\sum_{m=1}^M{\alpha_mG_m(x)}\right)$$

**对AdaBoost算法的说明**

步骤1中，假设数据集具有均匀的权值分布，即每个训练样本在基本分类器的学习中作用相同，这一假设保证1能够在原始数据上学习基本分类器$G_1(x)$

步骤2中，AdaBoost反复学习基本分类器，在每一轮$m=1,2,\cdots,M$顺次地执行下列操作：

&emsp;&emsp;(a) 使用当前分布$D_m$加权的训练数据集，学习基本分类器$G_m(x)$

&emsp;&emsp;(b) 计算基本分类器$G_m(x)$在加权训练集上的误差率：

$$e_m=P(G_m(x_i)\neq y_i)=\sum_{G_m(x_i)\neq y_i}w_{mi}$$
&emsp;&emsp;&emsp;&emsp;这里$w_{mi}$表示第$m$轮中第$i$个实例的权值，$\sum_{i=1}^Nw_{mi}=1$。这表明，$G_m(x)$在加权的训练数据集上的分类误差率是被$G_m(x)$误分类样本的权值之和，由此可以看出数据权值分布$D_m$与基本分类器$G_m(x)$的分类误差率的关系。

&emsp;&emsp;(c) 计算基本分类器$G_m(x)$的系数$\alpha_m$。$\alpha_m$表示$G_m(x)$在最终分类器中的重要性。当$\alpha_m \leq \frac{1}{2}$时，$\alpha_m \geq0$，并且$\alpha_m$随着$e_m$的减小而增大，所以误分类误差率越小的基本分类器在最终分类器中的作用越大。

&emsp;&emsp;(d) 更新训练数据集的权值分布为下一轮作准备，权值更新可以写成：

$$\begin{eqnarray}w_{m+1,i}=
\begin{cases}
\frac{w_{mi}}{Z_m}e^{-\alpha_m},&G_m(x_i)\neq y_i\cr\frac{w_{mi}}{Z_m}e^{\alpha_m},&G_m(x_i)=y_i
\end{cases}
\end{eqnarray}$$
&emsp;&emsp;&emsp;&emsp;由此可知，被基本分类器$G_m(x)$误分类样本的权值得以扩大，而被正确分类样本的权值却得以缩小。两相比较，误分类样本的权值被放大$e^{2\alpha_m}=\frac{e_m}{1-e_m}$倍，因此误分类样本在下一轮学习中起更大作用。不改变所给的训练数据，而不断改变训练数据权值的分布，使得训练数据在基本分类器的学习中器不同的作用，这是AdaBoost的一个特点。

步骤3中，线性组合$f(x)$实现$M$个基本分类器的加权表决。系数$\alpha_m$表示了基本分类器$G_m(x)$的重要性。这里，所有$\alpha_m$之和并不为1。$f(x)$的符号决定了实例$x$的类别，$f(x)$的绝对值表示分类的确信度。利用基本分类器的线性组合构建最终分类器是AdaBoost的另一个特点。


AdaBoost算法
+ 优点：泛化错误率低，易编码，可以应用在大部分分类器上，无参数调整
+ 缺点对离群点敏感
+ 适用数据类型：数值型和标称型数据

### 3 基于弱分类器单层决策树（决策树桩）构建AdaBoost

<span style='color: red'>单层决策树（decision stump，决策树桩）</span>是一种简单的决策树，仅基于单个特征来做决策。由于这棵树只有一个分裂过程，因此它实际上就是有一个树桩。

**单层决策树生成函数**
```
将最小错误率min_error设为正无穷
对数据集中的每一个特征（第一层循环）
    对每个步长（第二层循环）
        对每个不等号（第三层循环）
            建立一棵单层决策树并利用加权 数据集对它进行测试
            如果错误率低于min_error，则将当前单层决策树设为最佳单层决策树
返回最佳单层决策树
```

In [1]:
import numpy as np


def load_simp_data():
    '''A simple home_made data set.'''
    data_mat = np.matrix([[1., 2.1], [2., 1.1], [1.3, 1.], [1., 1.], [2., 1.]])
    class_labels = [1., 1., -1.0, -1.0, 1.0]
    return data_mat, class_labels

In [2]:
data_mat, class_labels = load_simp_data()

In [3]:
def stump_classify(data_mat, dimension, thresh_val, thresh_ineq):
    '''Classify the data by thresh_val and thresh_inequal.

    Datum at same side will be classified as -1, which decided on thresh_value
    and thresh_ineq, the other is +1. This classification can be done based
    on any feature(dimension).

    Args:
        data_mat: data set, a matrix
        dimension: feature index
        thresh_val: float, thresh value
        thresh_ineq: str, thresh inequal, less than or greater than
    Returns:
        an array, class labels
    '''
    ret_arr = np.ones((np.shape(data_mat)[0], 1))
    if thresh_ineq == 'lt':
        ret_arr[data_mat[:, dimension] <= thresh_val] = -1.0
    else:
        ret_arr[data_mat[:, dimension] > thresh_val] = -1.0
    return ret_arr


def build_stump(data_arr, class_labels, D):
    '''Find the best decision stump.
    Args:
        data_arr: data set
        class_labels: class labels
        D: weight vector
    Returns:
        the best decision stump, minimum error and the best class
    '''
    data_mat = np.mat(data_arr)
    label_mat = np.mat(class_labels).T
    m, n = np.shape(data_mat)
    num_steps = 10.0
    best_stump = {}
    best_class_est = np.mat(np.zeros((m, 1)))
    min_error = np.inf  # init error sum, to +definity
    for i in range(n):  # loop over all dimensions
        range_min = np.min(data_mat[:, i])
        range_max = np.max(data_mat[:, i])
        step_size = (range_max - range_min) / num_steps
        # loop over all range in current dimension
        for j in range(-1, int(num_steps)+1):
            for inequal in ['lt', 'rt']:  # go over less than and greater than
                thresh_val = range_min + float(j)*step_size
                # call stump classify
                predict_val = stump_classify(data_mat, i, thresh_val, inequal)
                err_arr = np.mat(np.ones((m, 1)))
                err_arr[predict_val == label_mat] = 0
                # calculate total error multiplied by D
                weighted_error = float(D.T * err_arr)
                # print("Split: dim %d, thresh %.2f, thresh ineqal: %s, the \
                      # weighted error is %.3f" %
                      # (i, thresh_val, inequal, weighted_error))
                if weighted_error < min_error:  # update
                    min_error = weighted_error
                    best_class_est = predict_val.copy()
                    best_stump['dim'] = i
                    best_stump['thresh'] = thresh_val
                    best_stump['inequal'] = inequal
    return best_stump, min_error, best_class_est

>  第一个函数stump_classify()是通过阈值比较对数据进行分类的。所有在阈值一边的数据会分到类别-1，而在另一边的数据分到+1。该函数通过函数过滤来实现，首先将返回数组的全部元素设置为1，然后将所有不满足不等式要求的元素设置为-1。可以基于数据集中任一元素进行比较 ，同时也可以将不等号在大于小于之间切换。

> 第二个函数build_stump将会遍历stump_classify()函数所有可能的输入值，并找到数据集上最佳的单层决策树。这里的最佳是基于数据的权重向量D来定义的。第一层循环在数据集的所有特征上遍历，第二层循环遍历步长，第三层循环是在大于和小于之间切换不等式。

> 这里是基于权重向量D而不是其他错误计算指标来评价分类器的。如果需要使用其他分类器的话，就需要考虑D上最佳分类器所定义的计算过程。

In [4]:
D = np.mat(np.ones((5, 1))/5)
build_stump(data_mat, class_labels, D)

({'dim': 0, 'inequal': 'lt', 'thresh': 1.3}, 0.2, array([[-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.]]))

**完整的AdaBoost算法的实现**
```
对每次迭代：
    利用build_stump()函数找到最佳的单层决策树
    将最佳单层决策树加入到单层决策树数组
    计算alpha
    计算新的权重向量D
    更新累计类别估计值
    如果错误率等于0.0，则退出循环
```

In [5]:
def adaBoost(data_arr, class_labels, num_iter=40):
    '''adaBoost algorithm(Adaptive boosting).
    Args:
        data_arr: data set
        class_labels: data set labels
        num_iter: the number of iterations
    Returns:
        a list containing weak learners.
    '''
    weak_class_arr = []  # a list containing weak learners
    m = np.shape(data_arr)[0]
    D = np.mat(np.ones((m, 1))/m)  # init weight vector to all equal
    agg_class_est = np.mat(np.zeros((m, 1)))
    for i in range(num_iter):
        # basic learner
        best_stump, error, class_est = build_stump(data_arr, class_labels, D)
        # print("D: ", D.T)
        # calculate alpha, throw in max(error,eps) to account for error=0
        alpha = 0.5 * np.log((1.0-error)/max(error, 1e-16))
        best_stump['alpha'] = alpha
        # store stump parameters in array
        weak_class_arr.append(best_stump)
        # print('class_est: ', class_est)
        # update weight vector
        # exponent for D calculation, getting messy
        expon = np.multiply(-1*alpha*np.mat(class_labels).T, class_est)
        D = np.multiply(D, np.exp(expon))
        D = D / np.sum(D)
        # calculate training error of all classifiers, if this is 0 quit for
        # loop early(use break)
        agg_class_est += alpha*class_est
        # print('agg_class_est: ', agg_class_est)
        agg_errors = np.multiply(np.sign(agg_class_est) !=
                                 np.mat(class_labels).T, np.ones((m, 1)))
        error_rate = agg_errors.sum() / m
        print('total error: ', error_rate, '\n')
        # exit if error_rate is equal to 0
        if error_rate == 0.0:
            break
    return weak_class_arr  # exit if maximum iterations

> AdaBoost算法的输入参数：数据集、类别标签以及迭代次数（是用户指定的参数）

> agg_class_est记录每个数据点的类别估计累计值

> 计算alpha时的max(error, 1e-16)用于确保在没有错误时不会发生除零溢出。

In [6]:
classifier_array = adaBoost(data_mat, class_labels, 9)

total error:  0.2 

total error:  0.2 

total error:  0.0 



In [7]:
classifier_array

[{'alpha': 0.69314718055994529, 'dim': 0, 'inequal': 'lt', 'thresh': 1.3},
 {'alpha': 0.9729550745276565, 'dim': 1, 'inequal': 'lt', 'thresh': 1.0},
 {'alpha': 0.89587973461402726,
  'dim': 0,
  'inequal': 'lt',
  'thresh': 0.90000000000000002}]

### 4 测试算法：基于AdaBoost分类函数

In [8]:
def ada_classify(data2class, classifier_arr):
    '''Use weak learners trained to do classification.
    Args:
        data2class: data set to classify
        classifier_arr: a list of weak learners
    Returns:
        a label matrix
    '''
    data_mat = np.mat(data2class)
    m = np.shape(data_mat)[0]
    agg_class_est = np.mat(np.zeros((m, 1)))
    for i in range(len(classifier_arr)):  # loop over all weak learners
        # call stump classify
        class_est = stump_classify(data_mat, classifier_arr[i]['dim'],
                                   classifier_arr[i]['thresh'],
                                   classifier_arr[i]['inequal'])
        agg_class_est += classifier_arr[i]['alpha'] * class_est
        # print(agg_class_est)
    return np.sign(agg_class_est)

In [9]:
classifier_arr = adaBoost(data_mat, class_labels, 30)  # 弱分类器集合

total error:  0.2 

total error:  0.2 

total error:  0.0 



In [10]:
ada_classify([0, 0], classifier_arr)  # 分类

matrix([[-1.]])

In [11]:
ada_classify([[5, 5], [0, 0]], classifier_arr)   # 分类

matrix([[ 1.],
        [-1.]])

### 5 实例

预测患有疝病的马能不能存活。

In [12]:
def load_data_set(file_name):
    '''General function to parse tab-delimited floats.'''
    # get the number of fields
    num_feat = len(open(file_name).readline().split('\t'))
    data_mat = []
    data_lab = []
    fr = open(file_name)
    for line in fr.readlines():
        line_arr = []
        cur_line = line.strip().split('\t')
        for i in range(num_feat-1):
            line_arr.append(float(cur_line[i]))
        data_mat.append(line_arr)
        data_lab.append(float(cur_line[-1]))
    return data_mat, data_lab

In [13]:
data_arr, label_arr = load_data_set('horseColicTraining2.txt')

In [14]:
classifier_arr = adaBoost(data_arr, label_arr, 10)

total error:  0.284280936455 

total error:  0.284280936455 

total error:  0.247491638796 

total error:  0.247491638796 

total error:  0.254180602007 

total error:  0.240802675585 

total error:  0.240802675585 

total error:  0.220735785953 

total error:  0.247491638796 

total error:  0.230769230769 



In [15]:
test_arr, test_label_arr = load_data_set('horseColicTest2.txt')

In [16]:
prediction10 = ada_classify(test_arr, classifier_arr)

In [17]:
err_arr = np.mat(np.ones((67, 1)))
err_arr[prediction10 != np.mat(test_label_arr).T].sum()

16.0

In [18]:
err_ratio = _ / 67  # error ratio
err_ratio

0.23880597014925373

### 6 总结

有文献称，对于表现好的数据集，AdaBoost的测试错误率就会达到一个稳定值，并不会随着分类器的增多而上升。

AdaBoost和SVM是监督学习中很强大的两种方法。两者有很多相似之处。可以把弱分类器看成SVM中的一个核函数，也可以按照最大化某个最小间隔的方式重写AdaBoost算法。而它们的不同之处在于其所定义的间隔计算方式有所不同，因此导致结果不同。特别是在高维空间下，两者的差异就会更加明显。